## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[["Attrition", "Department"]]

In [5]:
# Create a list of at least 10 column names to use as X data
selected_features = [
    "Age",
    "HourlyRate",
    "DistanceFromHome",
    "Education",
    "EnvironmentSatisfaction",
    "JobInvolvement",
    "JobLevel",
    "JobSatisfaction",
    "PerformanceRating",
    "TotalWorkingYears"
]

# Create X_df using your selected columns
X_df = attrition_df[selected_features]

# Show the data types for X_df
X_df.dtypes

Age                        int64
HourlyRate                 int64
DistanceFromHome           int64
Education                  int64
EnvironmentSatisfaction    int64
JobInvolvement             int64
JobLevel                   int64
JobSatisfaction            int64
PerformanceRating          int64
TotalWorkingYears          int64
dtype: object

In [6]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)

In [7]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train.info()
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1102 entries, 1343 to 1126
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   Age                      1102 non-null   int64
 1   HourlyRate               1102 non-null   int64
 2   DistanceFromHome         1102 non-null   int64
 3   Education                1102 non-null   int64
 4   EnvironmentSatisfaction  1102 non-null   int64
 5   JobInvolvement           1102 non-null   int64
 6   JobLevel                 1102 non-null   int64
 7   JobSatisfaction          1102 non-null   int64
 8   PerformanceRating        1102 non-null   int64
 9   TotalWorkingYears        1102 non-null   int64
dtypes: int64(10)
memory usage: 94.7 KB
<class 'pandas.core.frame.DataFrame'>
Index: 368 entries, 1041 to 1370
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   Age               

In [9]:
# Create a StandardScaler

scaler = StandardScaler()
# Fit the StandardScaler to the training data
X_train_scaled = scaler.fit_transform(X_train)

# Scale the training and testing data
X_test_scaled = scaler.transform(X_test)

In [11]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
dept_encoder = OneHotEncoder(sparse_output =False)

# Fit the encoder to the training data
y_train_dept = dept_encoder.fit_transform(y_train[["Department"]])

# Create two new variables by applying the encoder
# to the training and testing data
y_test_dept = dept_encoder.transform(y_test[["Department"]])

In [12]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
y_train_attr = attrition_encoder.fit_transform(y_train[["Attrition"]])

# Create two new variables by applying the encoder
# to the training and testing data
y_test_attr = attrition_encoder.transform(y_test[["Attrition"]])

## Part 2: Create, Compile, and Train the Model

In [13]:
# Find the number of columns in the X training data.

input_dim = X_train.shape[1]
print(f"Number of input features: {input_dim}")
# Create the input layer
input_layer = layers.Input(shape=(input_dim,))

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu')(shared_layer1)

Number of input features: 10


In [14]:
# Create a branch for Department
# with a hidden layer and an output layer
dept_hidden = layers.Dense(16, activation='relu')(shared_layer2)
# Create the hidden layer
dept_output = layers.Dense(y_train_dept.shape[1], activation='softmax', name='department')(dept_hidden)

# Create the output layer


In [15]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(16, activation='relu')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(y_train_attr.shape[1], activation='softmax', name='attrition')(attrition_hidden)

In [16]:
# Create the model

model = Model(inputs=input_layer, outputs=[dept_output, attrition_output])
# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'department': 'categorical_crossentropy',
        'attrition': 'categorical_crossentropy'
    },
    metrics=['accuracy']
)

# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │        704 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      2,080 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 16)        │        528 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 16)        │        528 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department (Dense)  │ (None, 3)         │         51 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition (Dense)   │ (None, 2)         │         34 │ dense_3[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,925 (15.33 KB)

 Trainable params: 3,925 (15.33 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
# Train the model
history = model.fit(
    X_train_scaled,
    {
        'department': y_train_dept,
        'attrition': y_train_attr
    },
    validation_split=0.2,  # optional
    epochs=100,
    batch_size=32
)

Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_accuracy: 0.9300 - attrition_loss: 0.1990 - department_accuracy: 0.8408 - department_loss: 0.4028 - loss: 0.6019 - val_attrition_accuracy: 0.7738 - val_attrition_loss: 0.6426 - val_department_accuracy: 0.5656 - val_department_loss: 1.1191 - val_loss: 1.7659
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_accuracy: 0.9079 - attrition_loss: 0.2265 - department_accuracy: 0.8433 - department_loss: 0.3980 - loss: 0.6246 - val_attrition_accuracy: 0.7783 - val_attrition_loss: 0.6357 - val_department_accuracy: 0.5792 - val_department_loss: 1.1485 - val_loss: 1.7890
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_accuracy: 0.9037 - attrition_loss: 0.2201 - department_accuracy: 0.8111 - department_loss: 0.4256 - loss: 0.6455 - val_attrition_accuracy: 0.7828 - val_attrition_loss: 0.6724 - val_department_accuracy: 0.5566 - val_department_loss: 1.1425 - val_loss: 1.8191
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━

In [23]:
# Evaluate the model with the testing data
eval_results = model.evaluate(X_test_scaled, [y_test_dept, y_test_attr])

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_accuracy: 0.8347 - attrition_loss: 1.1473 - department_accuracy: 0.5521 - department_loss: 2.6424 - loss: 3.7741  


In [24]:
# Print the accuracy for both department and attrition
print(f"\nTest Accuracy - Department: {eval_results[3]:.4f}")
print(f"Test Accuracy - Attrition: {eval_results[4]:.4f}")


Test Accuracy - Department: 0.8288
Test Accuracy - Attrition: 0.5679


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Class imbalance is going to mean that accuracy is probably not the best metric to use.
2.  We used binary output for attrition, because it is a simple yes/no, and a multiclass output for department because there is more than one possible output. Specifically, we used sigmoid and softmax
3. This could really benefit from Hyperparameter tuning.